<a href="https://colab.research.google.com/github/ZikrullaRaxmatov/Zako_AI_Lessons/blob/main/DataAnalysis_W1_6_Pandas3(df_from_API).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zako-ai

# Pandas `DataFrame`larni API so'rovlari yordamida yaratish
Ushbu misolda biz AQSh Geologiya Xizmatining API'sidan yer silkinishlari ma'lumotlari bo'lgan JSON ob'ektini olib, uni `pandas.DataFrame`ga aylantiramiz.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/


### Get Data from API

In [ ]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

200 javobi OK degan ma'noni anglatadi, shuning uchun biz natijadan ma'lumotlarni olishimiz mumkin. API'dan JSON yuklamasini so'raganimiz uchun, biz uni `json()` metodidan foydalanib, javobdan ajratib olishimiz mumkin.

### JSON Javobidan Ma'lumotni Ajratish
Ma'lumotlarimiz qaerda joylashganini bilish uchun javob ma'lumotlarining strukturasini tekshirishimiz kerak.


In [ ]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

USGS API bizning so'rovimiz haqida ma'lumotni `metadata` kalitida taqdim etadi. E'tibor bering, natijangiz tanlangan sana oralig'idan qat'iy nazar boshqacha bo'ladi, chunki API ma'lumotlarni qachon olib kelganligini ko'rsatuvchi vaqt belgisini o'z ichiga oladi.


In [ ]:
earthquake_json['metadata']

{'generated': 1604267813000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2020-10-01&endtime=2020-10-31',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.10.3',
 'count': 13706}

JSON massivining har bir elementi `features` - bu bizning DataFrame uchun bir qator ma'lumotni tashkil etadi.


In [ ]:
type(earthquake_json['features'])

list

Ma'lumotlaringiz, bu kodni qachon ishga tushirganingizga qarab, farq qilishi mumkin.


In [ ]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1,
  'place': '50 km ENE of Susitna North, Alaska',
  'time': 1604102395919,
  'updated': 1604103325550,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak020dz5f85a',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak020dz5f85a&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 15,
  'net': 'ak',
  'code': '020dz5f85a',
  'ids': ',ak020dz5f85a,',
  'sources': ',ak,',
  'types': ',origin,phase-data,',
  'nst': None,
  'dmin': None,
  'rms': 1.36,
  'gap': None,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.0 - 50 km ENE of Susitna North, Alaska'},
 'geometry': {'type': 'Point', 'coordinates': [-148.9807, 62.3533, 5]},
 'id': 'ak020dz5f85a'}

### DataFrame ga aylantirish
Biz `features` JSON massividagi har bir kirishdan `properties` bo'limini olishimiz kerak, shunda `DataFrame` yaratishimiz mumkin.


In [ ]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.00,"50 km ENE of Susitna North, Alaska",1604102395919,1604103325550,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak020dz5f85a,",",ak,",",origin,phase-data,",NaN,NaN,1.36,NaN,ml,earthquake,"M 1.0 - 50 km ENE of Susitna North, Alaska"
1,0.64,"15km NNE of Warner Springs, CA",1604102330860,1604114890185,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ci39451271,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",23.0,0.06759,0.16,123.0,ml,earthquake,"M 0.6 - 15km NNE of Warner Springs, CA"
2,1.09,"15km NNE of Warner Springs, CA",1604102075440,1604102309453,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ci39451255,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",39.0,0.06308,0.22,91.0,ml,earthquake,"M 1.1 - 15km NNE of Warner Springs, CA"
3,0.34,"15km NNE of Warner Springs, CA",1604101989680,1604102996061,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ci39451263,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",15.0,0.06228,0.15,129.0,ml,earthquake,"M 0.3 - 15km NNE of Warner Springs, CA"
4,0.59,"13km NE of Indio, CA",1604101849950,1604102079270,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ci39451231,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",17.0,0.09717,0.19,98.0,ml,earthquake,"M 0.6 - 13km NE of Indio, CA"


### (Ixtiyoriy) CSV faylga yozib qo'yish

In [ ]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>